In [1]:
import enchant
from PIL import Image, ImageEnhance
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
import cv2
import re 
import matplotlib.pyplot as plt
import numpy as np
from imgProcessor.measure.sharpness import parameters
from skimage.util import random_noise
# from skimage.exposure import is_low_contrast
# import skimage.measure
import imutils
from skimage.transform import rotate
from scipy.signal import convolve2d
import math
from skimage.restoration import estimate_sigma


In [2]:
def estimateNoise(img):
    return estimate_sigma(img, multichannel=True, average_sigmas=True)

def accuracyByLevenshteinDistance(textOCR,label):
    #tira espaços, tab e \n
    text = re.sub(r"[\n\t\s]*", "", textOCR)
    labelText = re.sub(r"[\n\t\s]*", "", label)
    
    #calcula a distancia de levenshtein entre os textos
    levDistance = enchant.utils.levenshtein(text, labelText)
    
    #calcula a acurácia fazendo accuracy = n-erros/n
    accuracy = (len(labelText)-levDistance) / len(labelText)
    return accuracy

def blurLevel(image):
    return cv2.Laplacian(image, cv2.CV_64F).var()

def instantiate_histogram():    
    histArray= []
    
    for i in range(0,256):
        histArray.append(str(i))
        histArray.append(0)
    
    hist = {histArray[i]: histArray[i + 1] for i in range(0, len(histArray), 2)} 
    
    return hist

def count_intensity_values(hist, img):
    for row in range(img.shape[0]):
        for column in range(img.shape[1]):
            hist[str(img[row][column])] += 1
     
    return hist

def plotHist(hist, hist2=''):
    if hist2 != '':
        figure, axarr = plt.subplots(1,2, figsize=(20, 10))
        axarr[0].bar(hist.keys(), hist.values())
        axarr[1].bar(hist2.keys(), hist2.values())
    else:
        plt.bar(hist.keys(), hist.values())
        plt.xlabel("Níveis intensidade")
        ax = plt.gca()
        ax.axes.xaxis.set_ticks([])
        plt.grid(True)
        plt.show()
        
def get_hist_proba(hist, n_pixels):
    hist_proba = {}
    for i in range(0, 256):
        hist_proba[str(i)] = hist[str(i)] / n_pixels
    
    return hist_proba

def getMediumBrightness(img):
    hist = instantiate_histogram()
    hist = count_intensity_values(hist, img)
    n = img.shape[0] * img.shape[1]
    histP = get_hist_proba(hist, n)
    
    L = img.max()
    mediumBright = 0
    
    for i in range(256):
        mediumBright += i*histP[str(i)]
        
    return mediumBright

def biggest_contour(contours):
        biggest = np.array([])
        max_area = 0
        for i in contours:
            area = cv2.contourArea(i)
            if area > 58000:
                peri = cv2.arcLength(i, True)
                approx = cv2.approxPolyDP(i, 0.015 * peri, True)
                if area > max_area and len(approx) == 4:
                    biggest = approx
                    max_area = area
        return biggest
 

def order_points(pts):
        rect = np.zeros((4, 2), dtype = "float32")

        s = np.sum(pts,axis = 1)

        rect[0] = pts[np.argmin(s)]
        rect[2] = pts[np.argmax(s)]
        
        diff = np.diff(pts, axis = 1)
        rect[3] = pts[np.argmin(diff)]
        rect[1] = pts[np.argmax(diff)]
     
        # return the ordered coordinates
        return rect
       
def getAngle(img):
    image = imutils.resize(img, height = 500)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    gray = cv2.bilateralFilter(gray, 20, 30, 30)
    edged = cv2.Canny(gray, 50, 150)
    
  
    cnts = None
    cnts, hierarchy  = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:10]

    
    screenCnt = None
    screenCnt = biggest_contour(cnts)
    
    if(len(screenCnt)==0 ):
        edged = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,13,0.5)
        cnts = None
        cnts, hierarchy  = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:10]
        
        screenCnt = None
        screenCnt = biggest_contour(cnts)

    if(len(screenCnt) >0 ):
        cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 1)
        screenCnt = order_points(screenCnt[:,0,:])
        angle = cv2.minAreaRect(screenCnt)[-1]
        angle = 90 - angle if (angle>45) else angle         
        
        return angle
    else:
        return None


def estimate_noise(I):
    
    I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    Image  = cv2.Sobel(src=I, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=5) #
    
    H, W = Image.shape

    M = [[1, -2, 1],
        [-2, 4, -2],
        [1, -2, 1]]

    sigma = np.sum(np.sum(np.absolute(convolve2d(Image, M))))

    sigma = sigma * (math.sqrt(math.pi/2) * (1 /  (6 * (W-2) * (H-2))))

    return sigma

In [21]:
def newEst(I):
    I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
   
    M = [[1, -2, 1],
        [-2, 4, -2],
        [1, -2, 1]]
    
    im = convolve2d(I, M)
    Image  = cv2.Sobel(src=im, ddepth=cv2.CV_32SC1, dx=1, dy=1, ksize=3)
    
    H, W = Image.shape
    sigma = np.sum(np.sum(np.absolute(Image)))

    sigma = sigma * (math.sqrt(math.pi/2) * (1 /  (6 * (W-2) * (H-2))))

    return sigma

In [77]:
import pandas as pd

In [3]:
df1 = pd.read_csv("accuracy_1.csv")
df2 = pd.read_csv("accuracy_2.csv")

accuracyAfter = pd.concat([df1, df2])
# print(accuracyAfter)

accAfter = np.asarray(accuracyAfter['Accuracy'].to_list(), dtype='float')
# print(accAfter)

NameError: name 'pd' is not defined

In [65]:
acc = np.array([])
high = np.array([])
low = np.array([])
rot = np.array([])

for i in range(0,200):
    # print("---IMAGE 1"+str(i).zfill(3)+"---")
    img = np.array([])
    img = cv2.imread('./images/1'+str(i).zfill(3) +'-receipt.jpg')
    
    if (i == 25 or i == 34 or i == 41 or i == 45 or i == 63 or i == 109 or i == 137 or i == 169 or i == 194 or i == 192): continue
    # name = '1' + str(i).zfill(3)
    # print(accuracyAfter['Image'].to_list())
    # if(int(name) not in accuracyAfter['Image'].to_list()): continue
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # altera o contraste com PILLOW
    im = Image.fromarray(img)
    enhancer = ImageEnhance.Contrast(im)
    #diminui o contraste 
    imLowContrast = enhancer.enhance(0.2)
    #aumenta o contraste 
    imHighContrast = enhancer.enhance(1.8)

    # #volta a img para o mesmo formato da original
    # # high = np.asarray(imHighContrast) 
    # # high= cv2.cvtColor(high, cv2.COLOR_RGB2BGR)

    # low = np.asarray(imLowContrast) 
    # low = cv2.cvtColor(low, cv2.COLOR_RGB2BGR)

    
    
    
    #altera o brilho com PILLOW
    # im2 = Image.fromarray(img)
    # enhancer2 = ImageEnhance.Brightness(im2)
    # #diminui o brilho
    # imLowBrightness = enhancer2.enhance(0.5)
    # #aumenta o brilho 
    # imHighBrightness = enhancer2.enhance(1.5)
    
    #volta a img para o mesmo formato da original
    imHighBrightness = np.asarray(imHighContrast) 
    imHighBrightness = cv2.cvtColor(imHighBrightness, cv2.COLOR_BAYER_BG2GRAY)

    imLowBrightness = np.asarray(imLowContrast) 
    imLowBrightness = cv2.cvtColor(imLowBrightness, cv2.COLOR_BAYER_BG2GRAY)
    
    # aplica efeito de blur com opencv
    # blur = cv2.GaussianBlur(img,(5,5),3)
    
    # aplica ruido gaussiano
    # noise_img = random_noise(img, mode='gaussian', var=0.2**2)
    # noise_img = (255*noise_img).astype(np.uint8)

    #rotaciona
    # rotated = rotate(img, 15, resize=True) * 255
    # rotated = (rotated).astype(np.uint8)
    
    #le a label salva
    label = open('./images/1'+str(i).zfill(3)+'-receipt.txt', 'r') 
    labelText = label.read()
    
    text = pytesseract.image_to_string(img)
    accuracyOriginal = accuracyByLevenshteinDistance(text,labelText)
    acc = np.append(acc,accuracyOriginal)
    
    textHigh = pytesseract.image_to_string(imHighBrightness, config='--psm 4')
    accuracyHigh = accuracyByLevenshteinDistance(textHigh,labelText)
    high = np.append(high,accuracyHigh)
    
    textLow = pytesseract.image_to_string(imLowBrightness, config='--psm 4')
    accuracyLow = accuracyByLevenshteinDistance(textLow,labelText)
    low = np.append(low,accuracyLow)
    
    # print("original " + str(getMediumBrightness(img)))
    # print("accuracy -> " + str(accuracyOriginal)) 
    # plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    # plt.show()
    
    # print("alto " + str(getMediumBrightness(imHighBrightness)))
    # print("accuracy -> " + str(accuracyHigh)) 
    # plt.imshow(cv2.cvtColor(imHighBrightness, cv2.COLOR_BGR2RGB))
    # plt.show()
    
    # print("baixo " + str(getMediumBrightness(imLowBrightness)))
    # print("accuracy -> " + str(accuracyLow)) 
    # plt.imshow(cv2.cvtColor(imLowBrightness, cv2.COLOR_BGR2RGB))
    # plt.show()
    
   
    # aplica efeito de blur com opencv
    # blur = cv2.GaussianBlur(img,(5,5),5)
    
    # aplica ruido gaussiano
    # noise_img = random_noise(img, mode='gaussian', var=0.2**2)
    # noise_img = (255*noise_img).astype(np.uint8)

    #rotaciona
    # rotated = rotate(img, 20, resize=True) * 255
    # rotated = (rotated).astype(np.uint8)
    
    #le a label salva
    # label = open('./images/1'+str(i).zfill(3)+'-receipt.txt', 'r') 
    # labelText = label.read()
    
    # imgBlurLevel = blurLevel(img)
    
    # noiseLevel = estimate_noise(img)
  
    # imgAngle = getAngle(img)
    # # print(imgAngle)
    # blur = np.append(blur, imgBlurLevel)
    # noise = np.append(noise, noiseLevel)
    # rot = np.append(rot, imgAngle)
    
    # text =''
    # text = pytesseract.image_to_string(img)
    # accuracyOriginal = accuracyByLevenshteinDistance(text,labelText)
    # accuracyBefore = np.append(accuracyBefore, accuracyOriginal)
    
    # print("original")
    # print("blur level -> " + str(imgBlurLevel))
    # print("angle -> " + str(imgAngle))
    # print("noise level -> " + str(noiseLevel))
    
    
    # print("accuracy -> " + str(accuracyOriginal)) 
    
    # plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    # plt.show()
    # plt.imshow(cv2.cvtColor(rotated, cv2.COLOR_BGR2RGB))
    # plt.show()
  
    
    # if(accuracyOriginal > 0.9):
    #     blurLGood = np.append(blurLGood, imgBlurLevel)
    #     noiseGood = np.append(noiseGood, noiseLevel)
    #     angleGood = np.append(angleGood, imgAngle)
        
    #     accuracyGood = np.append(accuracyGood, accuracyOriginal)   

    # else:
    #     blurLBad = np.append(blurLBad, imgBlurLevel)
    #     noiseBad = np.append(noiseBad, noiseLevel)
        
    #     if(imgAngle != None):
    #         angleBad = np.append(angleBad, imgAngle)
    
    # 1600 a 2000 20 a 30
    # >2100 >30 
    #
    
    #ruins descartadas naive
       
    # if((imgBlurLevel <= 108 and noiseLevel <= 10) or ((imgBlurLevel >= 1600 and imgBlurLevel < 2000 )and (noiseLevel >= 20 and noiseLevel < 30))
    #    or (imgBlurLevel > 2105 and noiseLevel >= 30)):
    #     if(imgAngle != None):
    #         if(imgAngle >= 0.6):
                     
    #             blurLGood = np.append(blurLGood, imgBlurLevel)
    #             noiseGood = np.append(noiseGood, noiseLevel)
    #             angleGood = np.append(angleGood, imgAngle)
                    
    #             accuracyGood = np.append(accuracyGood, accuracyOriginal)
                
    #     else:
    #         blurLGood = np.append(blurLGood, imgBlurLevel)
    #         noiseGood = np.append(noiseGood, noiseLevel)
                
    #         accuracyGood = np.append(accuracyGood, accuracyOriginal)
    # else:
    #     accuracyBad = np.append(accuracyBad, accuracyOriginal)   
       
       
    #boas - aceitaveis
    # if((imgBlurLevel < 350 and noiseLevel < 12) or (imgBlurLevel > 650 and noiseLevel > 25 and noiseLevel < 35)):
        
    #     acc = accuracyAfter['Accuracy'].where(accuracyAfter['Image'] == name).dropna()
    #     accuracyGood = np.append(accuracyGood, acc)
               
    # else:
    #     acc = accuracyAfter['Accuracy'].where(accuracyAfter['Image'] == name).dropna()
    #     accuracyBad = np.append(accuracyBad, acc)


                
    # print("blur")
    # print("blur level -> " + str(blurLevel(blur)))
    # # print("angle -> " + str(getAngle(blur)))
    # print("noise level -> " + str(estimate_noise(blur)))
    
    # textBlur = pytesseract.image_to_string(blur)
    # accuracyBlur = accuracyByLevenshteinDistance(textBlur,labelText)
    
    # print("accuracy -> " + str(accuracyBlur)) 
    
    # plt.imshow(cv2.cvtColor(blur, cv2.COLOR_BGR2RGB))
    # plt.show()
    
    # print("noise")
    # print("blur level -> " + str(blurLevel(noise_img)))
    # # print("angle -> " + str(getAngle(noise_img)))
    # print("noise level -> " + str(estimate_noise(noise_img)))
    
    # textNoise = pytesseract.image_to_string(noise_img)
    # accuracyNoise = accuracyByLevenshteinDistance(textNoise,labelText)
    
    # print("accuracy -> " + str(accuracyNoise)) 
    
    # plt.imshow(cv2.cvtColor(noise_img, cv2.COLOR_BGR2RGB))
    # plt.show()
    
    # print("rotated")
    # print("blur level -> " + str(blurLevel(rotated)))
    # print("angle -> " + str(getAngle(rotated)))
    # print("noise level -> " + str(estimateNoise(rotated)))
    
    # textRotated = pytesseract.image_to_string(rotated)
    # accuracyRotated = accuracyByLevenshteinDistance(textRotated,labelText)
    
    # print("accuracy -> " + str(accuracyRotated)) 
    
    # plt.imshow(cv2.cvtColor(rotated, cv2.COLOR_BGR2RGB))
    # plt.show()
# arquivo.close()
    
    



In [66]:
print(acc[:10])
print(high[:10])
print(low[:10])

m = np.array([])
for i in range(len(acc)):
    if(acc[i] <= low[i] or acc[i] <= high[i]):
        m = np.append(acc[i],m)
print(len(m))
print(len(acc))
# print(np.mean(acc))
# print(np.mean(high))
# print(np.median(low))

[0.62264151 0.         0.57966102 0.         0.80676329 0.33854167
 0.20689655 0.87896254 0.85140562 0.39936102]
[0.96698113 0.         0.06779661 0.0858209  0.87922705 0.1484375
 0.04597701 0.90489914 1.         0.17252396]
[0.93867925 0.         0.05423729 0.0858209  0.90821256 0.16666667
 0.         0.89913545 1.         0.        ]
92
190


In [91]:
0.4951 0 a 150
0.4632 0 a 100
0.5583 150
0.5041 160
0.4863 170
0.4694 180
0.4715 190
print("acima 90")
print(np.mean(blur[accAfter>0.9]))
print(np.mean(noise[accAfter>0.9]))

acima = rot[accAfter>0.9]
t = [not type(i) == type(None) for i in acima]
print(np.mean(acima[t]))

print("\nabaixo 15")
print(np.mean(blur[accAfter<0.15]))
print(np.mean(noise[accAfter<0.15]))

acima = rot[accAfter<0.15]
t = [not type(i) == type(None) for i in acima]
print(np.mean(acima[t]))
  

acima 90
885.5708433870394
14.20141360587874
0.5225926637649536

abaixo 15
460.1067304777127
9.887627171250585
nan


In [92]:
print("acima 90")
print(np.median(blur[accAfter>0.9]))
print(np.median(noise[accAfter>0.9]))

acima = rot[accAfter>0.9]
t = [not type(i) == type(None) for i in acima]
print(np.median(acima[t]))

print("\nabaixo 15")
print(np.median(blur[accAfter<0.15]))
print(np.median(noise[accAfter<0.15]))

acima = rot[accAfter<0.15]
t = [not type(i) == type(None) for i in acima]
print(np.median(acima[t]))
  

acima 90
675.561790750774
13.540825164212414
0.0

abaixo 15
460.1067304777127
9.887627171250585
nan
